# CHEM 1410 - Quantum Chemistry and Spectroscopy
## Particle in a Ramp Potential

by Prof. Geoff Hutchison, University of Pittsburgh

This notebook is designed for a demo -- how does the particle-in-a-box model change when the potential is a linear ramp instead of zero inside the box.

### Questions to Consider:

1. What happens to the **probability** of the first wavefunction when you increase the ramp slope? In particular, where is the peak of max probability? Where is it with the standard particle-in-a-box? Why is it different?
2. What about the **probabilities** with the second and third wave functions when you increase the ramp slope?
3. What happens to the **energy** of the wave functions when you increase the ramp slope?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import diags
from scipy.sparse.linalg import eigs
import ipywidgets as widgets
from ipywidgets import interact, FloatSlider, IntSlider, Dropdown
from IPython.display import display

%config InlineBackend.figure_format = 'retina'

In [ ]:
# constants in atomic units (ħ = m = 1)
ħ = 1.0
m = 1.0
N = 200

def solve_particle_in_ramp(L=1.0, k=10.0, n_show=3, probability=False):
    # spatial grid
    x = np.linspace(0, L, N)
    dx = x[1] - x[0]
    
    # kinetic energy (finite difference Laplacian)
    diag = np.ones(N)
    offdiag = np.ones(N-1)
    laplacian = (np.diag(-2*diag) + np.diag(offdiag,1) + np.diag(offdiag,-1)) / dx**2
    T = -(ħ**2)/(2*m) * laplacian
    
    # potential
    V = np.diag(k * x)
    
    # Hamiltonian
    H = T + V
    
    # solve eigenproblem
    energies, states = np.linalg.eigh(H)
    
    # normalize states
    for i in range(states.shape[1]):
        states[:,i] /= np.sqrt(np.trapezoid(np.abs(states[:,i])**2, x))
    
    # plot
    plt.figure(figsize=(7,5))

    for n in range(n_show):
        if probability:
            prob_density = np.abs(states[:,n])**2
            # scale for visibility
            if n_show > 1:
                scale = (energies[1] - energies[0]) * 0.8
            else:
                scale = energies[0] * 0.5 if energies[0] > 0 else 1.0
            prob_density = prob_density / prob_density.max() * scale
            plt.plot(x, prob_density + energies[n], label=f"n={n+1}, E={energies[n]:.3f}")
        else:  # "Wavefunction"
            plt.plot(x, states[:,n] + energies[n], label=f"n={n+1}, E={energies[n]:.3f}")
    
    plt.plot(x, np.diag(V), 'k--', alpha=0.6, label="V(x)")
    plt.xlabel("x")
    if probability:
        plt.ylabel(r"$|\psi(x)|^2$ (scaled) + Energy offset")
    else:
        plt.ylabel(r"$\psi(x)$ + Energy offset")
    plt.legend()
    plt.title(f"Particle in a Ramp Potential (slope={k:.2f})")
    plt.show()

# interactive widget
interact(
    solve_particle_in_ramp,
    probability=[('Probability', True), ('Wavefunction', False)],
    L=FloatSlider(value=1.0, min=0.5, max=5.0, step=0.1, description="Length"),
    k=FloatSlider(value=10.0, min=0.0, max=50.0, step=1.0, description="Ramp Slope"),
    n_show=IntSlider(value=3, min=1, max=6, step=1, description="# of States")
);
